In [1]:
from wfa.ml_logic.model import predict_new_images 
from wfa.utils.image_viz import plot_classified_images
from wfa.utils.get_new_images import get_new_image, split_tiles
from wfa.ml_logic import registry

2022-12-09 13:29:47.327015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 13:29:47.523624: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 13:29:47.523660: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 13:29:48.760195: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
adr = '45.2252929,5.6858077'
year1 = '2018' 
year2 = '2020'

new_img1 = get_new_image(adr, year1)
new_img2 = get_new_image(adr, year2)

X_test1 = split_tiles(new_img1)
X_test2 = split_tiles(new_img2)

import os
from dotenv import load_dotenv
load_dotenv()

new_model = registry.load_model()

cat_pred1 = predict_new_images(new_model, X_test1);
cat_pred2 = predict_new_images(new_model, X_test2);


Load model Production stage from mlflow...


2022-12-09 13:29:54.688210: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-09 13:29:54.688264: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-09 13:29:54.688293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schneider-1091-luis-roberto-sant-anna-henriques): /proc/driver/nvidia/version does not exist
2022-12-09 13:29:54.688584: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


4/4 [==============================] - 0s 23ms/step


In [3]:
import pandas as pd
import numpy as np

def summary(y_pred_class1:np.ndarray,
            y_pred_class2:np.ndarray):
    if y_pred_class1.shape != cat_pred2.shape:
         print('Images have different shapes, please check!')
    else:
        changes = pd.DataFrame(y_pred_class1 - y_pred_class2).applymap(lambda x: 1 if x != 0 else 0).to_numpy()
        
        inv_classes = {
            0:'AnnualCrop',
            1:'Forest', 
            2:'HerbaceousVegetation',
            3:'Highway',
            4:'Industrial',
            5:'Pasture',
            6:'PermanentCrop',
            7:'Residential',
            8:'River',
            9:'SeaLake',
            }
        pred1 = pd.DataFrame(y_pred_class1.reshape((y_pred_class1.shape[0]*y_pred_class1.shape[1]))).value_counts(normalize=True).rename_axis('cat_ID').reset_index(name='year1')
        pred2 = pd.DataFrame(y_pred_class2.reshape((y_pred_class2.shape[0]*y_pred_class2.shape[1]))).value_counts(normalize=True).rename_axis('cat_ID').reset_index(name='year2')

        summary = pred1.merge(pred2, how='outer')
        summary.fillna(0, inplace=True)
        summary['diff'] = summary['year2']-summary['year1']
        summary['cat_name'] = summary['cat_ID'].apply(lambda x: inv_classes[x])
        summary = summary[['cat_ID', 'cat_name', 'year1', 'year2', 'diff']].set_index('cat_ID')
        summary['year1'] = pd.Series(["{0:.0f}%".format(val * 100) for val in summary['year1']], index = summary.index)
        summary['year2'] = pd.Series(["{0:.0f}%".format(val * 100) for val in summary['year2']], index = summary.index)
        summary['diff'] = pd.Series(["{0:.0f}%".format(val * 100) for val in summary['diff']], index = summary.index)
        return changes, summary

In [4]:
changes, summary = summary(cat_pred1, cat_pred2)

In [5]:
changes

array([[0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 0, 0, 1, 1, 1, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 1]])

In [6]:
summary

,cat_name,year1,year2,diff
cat_ID,,,,
6,PermanentCrop,41%,22%,-19%
2,HerbaceousVegetation,15%,17%,2%
5,Pasture,13%,11%,-2%
9,SeaLake,11%,14%,3%
0,AnnualCrop,7%,23%,16%
1,Forest,5%,2%,-3%
4,Industrial,5%,9%,4%
8,River,2%,0%,-2%
3,Highway,1%,2%,1%
